# Combining datasets into one dataframe

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os.path
import itertools

## Create Index

The BBL number is going to be ideal for indexing as it is used for this purpose by the NYC databases. 

https://www.pincusco.com/data-dictionary/bbl-also-known-as-borough-block-and-lot-or-property-id/


https://www.propertyshark.com/Real-Estate-Reports/2018/12/13/all-you-need-to-know-about-nycs-bbl/#:~:text=BBL%20stands%20for%20borough%2C%20block,1%20%3D%20Manhattan

In [38]:
# Import the sales data from pckl
sales_pckl = '../data/sales_combined.pkl'

# This is the path to retrieve files one above the github directory
above = '../../../'
# Import valuation data using the above directory
valuation_pckl = above + 'assessment_og.pkl'

# Read the datasets into dataframes
sales = pd.read_pickle(sales_pckl)
valuation = pd.read_pickle(valuation_pckl)

In [40]:
# Add the BBL column for indexing to the valuation data
valuation['bbl'] = valuation['BORO'].astype(str) + valuation['BLOCK'].astype(str) + valuation['LOT'].astype(str)

# Add the BBL column for indexing to the sales data
sales['bbl'] = sales['borough'].astype(str) + sales['block'].astype(str) + sales['lot'].astype(str)

# Standardizing Columns and a bit of 

In [49]:
valuation.PERIOD.unique()

array(['FINAL'], dtype=object)